In [21]:
import pandas as pd
import numpy as np
import os
import gzip
import shutil
import geopandas as gpd
import pyspark as py
from pyspark import SparkContext
import csv
import json
import glob

In [ ]:
# get all directories
data_directory = r'./safegraph_data/'
allfile = []
for subdir, dirs, files  in os.walk(data_directory):
    for filename in files:
        filepath = subdir + os.sep + filename
        allfile.append(filepath)

In [ ]:
# unzip files
for file in allfile:
    _,_,year,month,_,file_name = file.split('/')
    file_name = file_name[:-3]
    # Change month here
    if (int(year) != 2021 and int(month)>2 and int(month)<6) \ 
        or (int(year)==2020 and int(month)>=1 and int(month)=<12) \
            or (int(year)==2021 and int(month)!=4):
        
        with gzip.open(file, 'rb') as f_in:
            with open('./data/'+file_name, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

In [22]:
geo = gpd.read_file('./CBG_data/data/cbg.geojson')
geo.head()

,StateFIPS,CountyFIPS,TractCode,BlockGroup,CensusBlockGroup,State,County,ClassCode,geometry
0,01,081,041600,1,010810416001,AL,Lee County,H1,"MULTIPOLYGON (((-85.37282 32.63424, -85.37275 ..."
1,01,081,041600,2,010810416002,AL,Lee County,H1,"MULTIPOLYGON (((-85.38346 32.64838, -85.38301 ..."
2,01,081,041700,4,010810417004,AL,Lee County,H1,"MULTIPOLYGON (((-85.37139 32.60139, -85.37138 ..."
3,01,073,011107,4,010730111074,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.64797 33.59205, -86.64771 ..."
4,01,073,011108,4,010730111084,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.65206 33.59869, -86.65204 ..."


In [23]:
geo = geo[geo['State']=='NY']

In [24]:
nyc_county = ['Bronx County','New York County','Kings County','Queens County','Richmond County']
geo_nyc = geo[geo.County.isin(nyc_county)]

In [25]:
geo_nyc.head()

,StateFIPS,CountyFIPS,TractCode,BlockGroup,CensusBlockGroup,State,County,ClassCode,geometry
127690,36,005,017500,2,360050175002,NY,Bronx County,H6,"MULTIPOLYGON (((-73.91570 40.83054, -73.91485 ..."
127691,36,005,014100,1,360050141001,NY,Bronx County,H6,"MULTIPOLYGON (((-73.91661 40.82499, -73.91592 ..."
127692,36,005,014500,1,360050145001,NY,Bronx County,H6,"MULTIPOLYGON (((-73.90584 40.83106, -73.90505 ..."
127693,36,005,007500,2,360050075002,NY,Bronx County,H6,"MULTIPOLYGON (((-73.91035 40.81995, -73.91022 ..."
127694,36,005,041800,1,360050418001,NY,Bronx County,H6,"MULTIPOLYGON (((-73.86287 40.89515, -73.86146 ..."


In [26]:
cbg_nyc = list(geo_nyc.CensusBlockGroup.unique())

In [27]:
sc = py.SparkContext()

In [28]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [29]:
files = glob.glob('./data/*.csv')
# df_social = pd.DataFrame()
# for f in files:
#     csv = pd.read_csv(f)
#     df_social = df_social.append(csv)

In [30]:
test_df1 = pd.read_csv('./data/2020-04-01-social-distancing.csv')
test_df1.shape

(219312, 20)

In [31]:
test_df2 = pd.read_csv('./data/2019-04-01-social-distancing.csv')
test_df2.head()

,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,...,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home,mean_home_dwell_time,mean_non_home_dwell_time,mean_distance_traveled_from_home
0,10330210004,2019-04-01T00:00:00-05:00,2019-04-02T00:00:00-05:00,84,3513,"{""16001-50000"":24,""0"":19,"">50000"":8,""2001-8000...","{""16001-50000"":51,"">50000"":112,""<1000"":76,""200...",21,593,"{""721-1080"":18,""361-720"":19,""61-360"":10,""<60"":...",...,"{""281419503001"":1,""281419502003"":1,""2800395050...",2,160,220,"{""21-45"":6,""481-540"":3,""541-600"":3,""46-60"":6,""...",67,"{""0-25"":20,""76-100"":38,""51-75"":15,""26-50"":11}",572,260,10218
1,10730049022,2019-04-01T00:00:00-05:00,2019-04-02T00:00:00-05:00,262,3150,"{""16001-50000"":20,""0"":119,"">50000"":33,""2001-80...","{""16001-50000"":62,"">50000"":29,""<1000"":74,""2001...",119,137,"{""721-1080"":30,""361-720"":32,""61-360"":49,""<60"":...",...,"{""010730102002"":1,""132150004003"":1,""3904101175...",2,12,1154,"{""21-45"":13,""481-540"":4,""541-600"":7,""46-60"":2,...",90,"{""0-25"":91,""76-100"":145,""51-75"":14,""26-50"":9}",365,174,18483
2,11210118001,2019-04-01T00:00:00-05:00,2019-04-02T00:00:00-05:00,89,1727,"{""16001-50000"":3,""0"":27,"">50000"":5,""2001-8000""...","{""16001-50000"":125,"">50000"":209,""<1000"":32,""20...",29,592,"{""721-1080"":16,""361-720"":17,""61-360"":13,""<60"":...",...,"{""011150405022"":1,""011210114001"":2,""0112101180...",3,85,276,"{""21-45"":1,""481-540"":3,""541-600"":3,""46-60"":2,""...",84,"{""0-25"":20,""76-100"":47,""51-75"":14,""26-50"":5}",650,213,6529
3,11250106021,2019-04-01T00:00:00-05:00,2019-04-02T00:00:00-05:00,442,4456,"{""16001-50000"":133,""0"":132,"">50000"":36,""2001-8...","{""16001-50000"":55,"">50000"":70,""<1000"":117,""200...",134,620,"{""721-1080"":88,""361-720"":100,""61-360"":71,""<60""...",...,"{""010730102002"":13,""011170302131"":1,""010730142...",16,144,1144,"{""21-45"":23,""481-540"":28,""541-600"":9,""46-60"":3...",74,"{""0-25"":92,""76-100"":221,""51-75"":102,""26-50"":24}",588,261,12612
4,21880002003,2019-04-01T00:00:00-08:00,2019-04-02T00:00:00-08:00,13,305,"{""1-1000"":3,"">50000"":4,""0"":6}","{"">50000"":36,""<1000"":313}",5,0,"{""721-1080"":1,"">1080"":3,""<60"":8}",...,"{""021880002003"":9,""021880002001"":2,""4208503180...",1,257,38,"{""46-60"":1,""721-840"":1,""1201-1320"":1,""301-360""...",0,"{""0-25"":5,""76-100"":5,""51-75"":1}",466,417,98774


In [32]:
test_df3 = pd.read_csv('./data/2021-03-01-social-distancing.csv')
test_df3.head()

,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,...,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home,mean_home_dwell_time,mean_non_home_dwell_time,mean_distance_traveled_from_home
0,10139533002,2021-03-01T00:00:00-06:00,2021-03-02T00:00:00-06:00,69,13950,"{""16001-50000"":16,""0"":16,"">50000"":8,""1-1000"":2...","{""16001-50000"":11,"">50000"":39,""<1000"":717,""100...",15,783,"{""721-1080"":16,""361-720"":15,""61-360"":5,""<60"":8...",...,"{""011010053021"":2,""011010022021"":2,""0110100590...",8,123,101,"{""21-45"":2,""481-540"":2,""541-600"":2,""46-60"":2,""...",79,"{""0-25"":7,""76-100"":33,""51-75"":13,""26-50"":8}",774,297,25165
1,10210601014,2021-03-01T00:00:00-06:00,2021-03-02T00:00:00-06:00,173,2527,"{""16001-50000"":25,""0"":25,"">50000"":24,""2001-800...","{""16001-50000"":50,"">50000"":100,""<1000"":228,""20...",25,677,"{""721-1080"":43,""361-720"":32,""61-360"":19,""<60"":...",...,"{""010730111091"":1,""010730144133"":1,""0110100510...",4,335,282,"{""21-45"":7,""481-540"":11,""541-600"":22,""46-60"":3...",59,"{""0-25"":48,""76-100"":65,""51-75"":42,""26-50"":18}",625,397,11082
2,10950303005,2021-03-01T00:00:00-06:00,2021-03-02T00:00:00-06:00,81,8172,"{""16001-50000"":16,""0"":7,"">50000"":4,""2001-8000""...","{""16001-50000"":43,"">50000"":101,""<1000"":236,""20...",11,773,"{""721-1080"":21,""361-720"":18,""61-360"":6,""<60"":7...",...,"{""011030051012"":1,""010950303004"":1,""0108900200...",3,288,152,"{""21-45"":4,""481-540"":6,""541-600"":3,""46-60"":3,""...",72,"{""0-25"":8,""76-100"":39,""51-75"":26,""26-50"":8}",782,331,16216
3,11010055032,2021-03-01T00:00:00-06:00,2021-03-02T00:00:00-06:00,100,4045,"{""16001-50000"":7,""0"":23,"">50000"":11,""2001-8000...","{""16001-50000"":205,"">50000"":12,""<1000"":223,""20...",24,895,"{""721-1080"":32,""361-720"":13,""61-360"":6,""<60"":1...",...,"{""011010053021"":3,""011010054033"":2,""0108723200...",5,134,161,"{""21-45"":3,""481-540"":5,""541-600"":3,""46-60"":3,""...",83,"{""0-25"":11,""76-100"":59,""51-75"":16,""26-50"":4}",819,260,5728
4,40210002061,2021-03-01T00:00:00-07:00,2021-03-02T00:00:00-07:00,479,2217,"{""16001-50000"":91,""0"":136,"">50000"":35,""2001-80...","{""16001-50000"":67,"">50000"":51,""<1000"":28,""2001...",143,770,"{""721-1080"":94,""361-720"":57,""61-360"":47,""<60"":...",...,"{""040134211022"":1,""040210002053"":1,""0402100020...",24,98,1066,"{""21-45"":30,""481-540"":20,""541-600"":17,""46-60"":...",82,"{""0-25"":111,""76-100"":251,""51-75"":94,""26-50"":21}",706,275,5034


In [33]:
bucket,visits = zip(*json.loads(test_df1.bucketed_percentage_time_home[20000]).items())
bucket

('26-50', '51-75', '0-25', '>100', '76-100')

In [34]:
bucket,visits = zip(*json.loads(test_df2.bucketed_percentage_time_home[60000]).items())
bucket

('0-25', '76-100', '51-75', '26-50')

In [35]:
bucket,visits = zip(*json.loads(test_df3.bucketed_percentage_time_home[430]).items())
bucket

('0-25', '76-100', '51-75', '26-50')

In [36]:
column_test_1 = test_df1.columns.to_list()

In [37]:
column_test_2 = test_df2.columns.to_list()

In [38]:
missing_column = []
for i in column_test_2:
    if i not in column_test_1:
        print(i)
        missing_column.append(test_df2.columns.get_loc(i))

mean_home_dwell_time
mean_non_home_dwell_time
mean_distance_traveled_from_home


In [39]:
missing_column

[20, 21, 22]

In [40]:
columns = test_df2.columns.to_list()
len(columns)

23

In [41]:
expand_list = ['bucketed_distance_traveled','median_dwell_at_bucketed_distance_traveled',\
               'bucketed_home_dwell_time','bucketed_away_from_home_time','bucketed_percentage_time_home']
expand_index=[]
for i in expand_list:
    expand_index.append(test_df1.columns.get_loc(i))
expand_index

[5, 6, 9, 17, 19]

In [42]:
files[0].split('/')[2].split('-')[0]

'2020'

In [43]:
def get_medians(lines):
#     try:
#         lines.append(dwellTime(lines[9]))
#     except:
#         lines.append(np.nan)
#     try:
#         lines.append(dwellTime(lines[6]))
#     except:
#         lines.append(np.nan)
#     try:
#         lines.append(dwellTime(lines[5]))
#     except:
#         lines.append(np.nan)
    lines.append(np.nan)
    lines.append(np.nan)
    lines.append(np.nan)
    return lines
# test1 = '{"721-1080":16,"361-720":15,"61-360":5,"<60":8,">1080":23}'
# def dwellTime(dwells):
#     import itertools
#     import bisect
#     bucket, visits = zip(*json.loads(dwells).items())
#     cdf = list(itertools.accumulate(visits))
#     return cdf[-1]//2
# dwellTime(test1)


In [44]:
for i,v in enumerate(columns):
    print((i,v))

(0, 'origin_census_block_group')
(1, 'date_range_start')
(2, 'date_range_end')
(3, 'device_count')
(4, 'distance_traveled_from_home')
(5, 'bucketed_distance_traveled')
(6, 'median_dwell_at_bucketed_distance_traveled')
(7, 'completely_home_device_count')
(8, 'median_home_dwell_time')
(9, 'bucketed_home_dwell_time')
(10, 'at_home_by_each_hour')
(11, 'part_time_work_behavior_devices')
(12, 'full_time_work_behavior_devices')
(13, 'destination_cbgs')
(14, 'delivery_behavior_devices')
(15, 'median_non_home_dwell_time')
(16, 'candidate_device_count')
(17, 'bucketed_away_from_home_time')
(18, 'median_percentage_time_home')
(19, 'bucketed_percentage_time_home')
(20, 'mean_home_dwell_time')
(21, 'mean_non_home_dwell_time')
(22, 'mean_distance_traveled_from_home')


In [45]:
test_df2.bucketed_away_from_home_time[1]

'{"21-45":13,"481-540":4,"541-600":7,"46-60":2,"721-840":2,"1201-1320":2,"301-360":10,"<20":145,"61-120":9,"241-300":11,"121-180":14,"421-480":4,"1321-1440":2,"841-960":4,"1081-1200":3,"961-1080":3,"601-660":2,"181-240":6,"661-720":4,"361-420":6}'

In [46]:
# create column name for new columns (range of col5, col9, col,19)
col_new = columns.copy()
bucket,visits = zip(*json.loads(test_df2.bucketed_distance_traveled[100]).items())
bucket = list(bucket)
del bucket [2]
for i in bucket:
    col_new.append('distance_traveled:'+i)

bucket,visits = zip(*json.loads(test_df2.bucketed_home_dwell_time[10000]).items())
for i in bucket:
    col_new.append('home_dwell_time:'+i)

# bucket,visits = zip(*json.loads(test_df2.bucketed_percentage_time_home[10000]).items())
# for i in bucket:
#     col_new.append('percentage_time_home:'+i)
col_new

['origin_census_block_group',
 'date_range_start',
 'date_range_end',
 'device_count',
 'distance_traveled_from_home',
 'bucketed_distance_traveled',
 'median_dwell_at_bucketed_distance_traveled',
 'completely_home_device_count',
 'median_home_dwell_time',
 'bucketed_home_dwell_time',
 'at_home_by_each_hour',
 'part_time_work_behavior_devices',
 'full_time_work_behavior_devices',
 'destination_cbgs',
 'delivery_behavior_devices',
 'median_non_home_dwell_time',
 'candidate_device_count',
 'bucketed_away_from_home_time',
 'median_percentage_time_home',
 'bucketed_percentage_time_home',
 'mean_home_dwell_time',
 'mean_non_home_dwell_time',
 'mean_distance_traveled_from_home',
 'distance_traveled:16001-50000',
 'distance_traveled:0',
 'distance_traveled:2001-8000',
 'distance_traveled:1-1000',
 'distance_traveled:1001-2000',
 'distance_traveled:8001-16000',
 'home_dwell_time:721-1080',
 'home_dwell_time:361-720',
 'home_dwell_time:61-360',
 'home_dwell_time:<60',
 'home_dwell_time:>1080']

In [47]:
bucket,visits = zip(*json.loads(test_df2.bucketed_distance_traveled[100]).items())
bucket = list(bucket)
del bucket[2]
bucket

['16001-50000', '0', '2001-8000', '1-1000', '1001-2000', '8001-16000']

In [48]:
def expand_range(lines):
    
    dt_key,distance_traveled = zip(*json.loads(lines[5]).items())
    if len(dt_key) == 7:
        distance_traveled = list(distance_traveled)
        del distance_traveled[2]
        for i in distance_traveled:
            lines.append(i)
    else:
        for i in distance_traveled:
            lines.append(i)
    
    dht_key,home_dwell_time = zip(*json.loads(lines[9]).items())
    if len(dht_key) != 5:
        for i in range(5):
            lines.append(np.nan)
    else:
        for i in home_dwell_time:
            lines.append(i)
        
#     pth_key,percentage_time_home = zip(*json.loads(lines[19]).items())
#     if len(pth_key) != 4:
#         for i in range(4):
#             lines.append(np.nan)
#     else:
#         for i in home_dwell_time:
#             lines.append(i)
    return lines
    

In [49]:
# # test code 
# import csv
# rdd2 = sc.textFile('./data/2020-04-01-social-distancing.csv')\
#     .map(lambda x: next(csv.reader([x])))\
#     .filter(lambda x: x[0] !='origin_census_block_group')\
#     .filter(lambda x: x[0] in cbg_nyc)\
#     .map(get_medians)\
#     .map(expand_range).collect()
# output = pd.DataFrame(rdd2,columns=col_new)
# output.iloc[:,18:]

# # old_method
# files_nyc = glob.glob('./nyc_data/*.csv')
# files_nyc = sorted(files_nyc)
# df_social = pd.DataFrame()
# for f in files:
#     csv = pd.read_csv(f)
#     df_social = df_social.append(csv)

In [70]:
files = sorted(files)
df_social = pd.DataFrame(columns=col_new)
df_social.to_csv('social_distancing.csv')

In [71]:
def write_file(rdd,file):
    output = pd.DataFrame(rdd,columns=col_new)
    print(str(output.shape)+' {}'.format(file.split('/')[2]))
    output.to_csv('social_distancing.csv',mode='a',header=False)

In [72]:
for file in files:
    year = file.split('/')[2].split('-')[0]
    num_col = len(list(enumerate(next(open(file, 'r')).strip().split(','))))
    if int(year) == 2020 and num_col < 23:
        rdd = sc.textFile(file)\
            .map(lambda x: next(csv.reader([x])))\
            .filter(lambda x: x[0] !='origin_census_block_group')\
            .filter(lambda x: x[0] in cbg_nyc)\
            .map(get_medians)\
            .map(expand_range).collect()
        write_file(rdd,file)
    else:
        rdd = sc.textFile(file)\
            .map(lambda x: next(csv.reader([x])))\
            .filter(lambda x: x[0] !='origin_census_block_group')\
            .filter(lambda x: x[0] in cbg_nyc)\
            .map(expand_range).collect()
        write_file(rdd,file)

(6289, 34) 2019-03-01-social-distancing.csv
(6288, 34) 2019-03-02-social-distancing.csv
(6288, 34) 2019-03-03-social-distancing.csv
(6287, 34) 2019-03-04-social-distancing.csv
(6288, 34) 2019-03-05-social-distancing.csv
(6285, 34) 2019-03-06-social-distancing.csv
(6285, 34) 2019-03-07-social-distancing.csv
(6282, 34) 2019-03-08-social-distancing.csv
(6278, 34) 2019-03-09-social-distancing.csv
(6279, 34) 2019-03-10-social-distancing.csv
(6287, 34) 2019-03-11-social-distancing.csv
(6287, 34) 2019-03-12-social-distancing.csv
(6286, 34) 2019-03-13-social-distancing.csv
(6286, 34) 2019-03-14-social-distancing.csv
(6287, 34) 2019-03-15-social-distancing.csv
(6278, 34) 2019-03-16-social-distancing.csv
(6278, 34) 2019-03-17-social-distancing.csv
(6285, 34) 2019-03-18-social-distancing.csv
(6282, 34) 2019-03-19-social-distancing.csv
(6281, 34) 2019-03-20-social-distancing.csv
(6277, 34) 2019-03-21-social-distancing.csv
(6278, 34) 2019-03-22-social-distancing.csv
(6280, 34) 2019-03-23-social-dis

(6276, 34) 2021-01-04-social-distancing.csv
(6289, 34) 2021-01-05-social-distancing.csv
(6298, 34) 2021-01-06-social-distancing.csv
(6291, 34) 2021-01-07-social-distancing.csv
(6291, 34) 2021-01-08-social-distancing.csv
(6283, 34) 2021-01-09-social-distancing.csv
(6288, 34) 2021-01-10-social-distancing.csv
(6277, 34) 2021-01-11-social-distancing.csv
(6284, 34) 2021-01-12-social-distancing.csv
(6281, 34) 2021-01-13-social-distancing.csv
(6274, 34) 2021-01-14-social-distancing.csv
(6270, 34) 2021-01-15-social-distancing.csv
(6265, 34) 2021-01-16-social-distancing.csv
(6267, 34) 2021-01-17-social-distancing.csv
(6280, 34) 2021-01-18-social-distancing.csv
(6271, 34) 2021-01-19-social-distancing.csv
(6266, 34) 2021-01-20-social-distancing.csv
(6268, 34) 2021-01-21-social-distancing.csv
(6256, 34) 2021-01-22-social-distancing.csv
(6253, 34) 2021-01-23-social-distancing.csv
(6259, 34) 2021-01-24-social-distancing.csv
(6267, 34) 2021-01-25-social-distancing.csv
(6247, 34) 2021-01-26-social-dis

In [73]:
df = pd.read_csv('social_distancing.csv')

In [74]:
 df.isna().sum()

Unnamed: 0                                         0
origin_census_block_group                          0
date_range_start                                   0
date_range_end                                     0
device_count                                       0
distance_traveled_from_home                      149
bucketed_distance_traveled                         0
median_dwell_at_bucketed_distance_traveled       356
completely_home_device_count                       0
median_home_dwell_time                             0
bucketed_home_dwell_time                           0
at_home_by_each_hour                               0
part_time_work_behavior_devices                    0
full_time_work_behavior_devices                    0
destination_cbgs                                   0
delivery_behavior_devices                          0
median_non_home_dwell_time                         0
candidate_device_count                             0
bucketed_away_from_home_time                  

In [77]:
check_2020 = df[(df['date_range_start']>'2019-05-31T00:00:00-05:00')&(df['date_range_start']<'2021-03-31T00:00:00-05:00')]

In [79]:
check_2020[['date_range_start']]['date_range_start'].unique()

array(['2020-03-01T00:00:00-05:00', '2020-03-02T00:00:00-05:00',
       '2020-03-03T00:00:00-05:00', '2020-03-04T00:00:00-05:00',
       '2020-03-05T00:00:00-05:00', '2020-03-06T00:00:00-05:00',
       '2020-03-07T00:00:00-05:00', '2020-03-08T00:00:00-05:00',
       '2020-03-09T00:00:00-04:00', '2020-03-10T00:00:00-04:00',
       '2020-03-11T00:00:00-04:00', '2020-03-12T00:00:00-04:00',
       '2020-03-13T00:00:00-04:00', '2020-03-14T00:00:00-04:00',
       '2020-03-15T00:00:00-04:00', '2020-03-16T00:00:00-04:00',
       '2020-03-17T00:00:00-04:00', '2020-03-18T00:00:00-04:00',
       '2020-03-19T00:00:00-04:00', '2020-03-20T00:00:00-04:00',
       '2020-03-21T00:00:00-04:00', '2020-03-22T00:00:00-04:00',
       '2020-03-23T00:00:00-04:00', '2020-03-24T00:00:00-04:00',
       '2020-03-25T00:00:00-04:00', '2020-03-26T00:00:00-04:00',
       '2020-03-27T00:00:00-04:00', '2020-03-28T00:00:00-04:00',
       '2020-03-29T00:00:00-04:00', '2020-03-30T00:00:00-04:00',
       '2020-03-31T00:00:

In [82]:
check_2020[['date_range_start']]['date_range_start'].unique().shape

(182,)